In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Preprocessing

In [ ]:
!pip install emot --upgrade
# to convert emojis to text
!pip install emoji
#to expand a contracted words
!pip install demoji
!pip install contractions

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from sklearn.naive_bayes import MultinomialNB
from textblob import TextBlob
import emot
import nltk.data
import numpy as np
import pandas as pd
import re
import time
import string
import emoji
import demoji
import contractions
import nltk
nltk.download('omw-1.4')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
pd.set_option("max_colwidth" ,220)
wn = nltk.WordNetLemmatizer()
nltk.download('wordnet')
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

english_stopwords = stopwords.words("english")
with open('/content/drive/MyDrive/hasoc/task4/assam/assamese.txt', encoding = 'utf-8') as f:
    assam_stopwords = f.readlines()
    for i in range(len(assam_stopwords)):
        assam_stopwords[i] = re.sub('\n','', assam_stopwords[i])
stopwords = english_stopwords + assam_stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
demoji.download_codes()
def emo(text):
  temp=emoji.demojize(text,delimiters=(" "," "))
  temp=temp.replace("_","  ")
  return temp

<ipython-input-37-e00aa54160fe>:1: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


In [ ]:
demoji.download_codes()
def emo(text):
  try:
   temp=emoji.demojize(str(text),delimiters=(" "," "))
   temp=temp.replace("_","  ")
   return temp
  except IndexError:
        return ""

<ipython-input-38-82a5fb4117d1>:1: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


In [ ]:
punct="!#$%&\'()*+,-.’’/:;<=>?@[\\]^_`{|}~’“‘"

In [ ]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', str(text))

In [ ]:
def clean_text(text):
    text = re.sub('@[a-zA-Z0-9]*', '', str(text))#to remove @ and its following word
    text = contractions.fix(text, slang=True)
    text = "".join([word.lower() for word in text if word not in string.punctuation]) #to remove punctuation
    text="".join([word.lower() for word in text if word not in punct])
    text = "".join([word for word in text if not word.isdigit()])#to remove digit
    text = " ".join(word for word in text.split() if word not in stopwords)
    return text

In [ ]:
import pandas as pd

In [ ]:
asa_train=pd.read_csv('/content/drive/MyDrive/hasoc/task4/assam/train_A_AH_HASOC2023.csv')
asa_test=pd.read_csv('/content/drive/MyDrive/hasoc/task4/assam/test_A_AH_HASOC2023.csv')

In [ ]:
asa_train['task_1'].value_counts()

HOF    2347
NOT    1689
Name: task_1, dtype: int64

In [ ]:
asa_train.shape,asa_test.shape

((4036, 3), (1009, 2))

In [ ]:
asa_train['clean_text'] = asa_train['text'].apply(lambda x:emo(x))
asa_train['clean_text'] = asa_train['clean_text'].apply(lambda x:remove_urls(x))
asa_train['clean_text'] = asa_train['clean_text'].apply(lambda x:clean_text(x))

In [ ]:
asa_train['clean_text']

0                                                                                                                                      লাগিলে ইযাৰ পৰাই কলি লাগাম
1                                                                         আমি গেদা মই গেদা rolling floor laughing rolling floor laughing rolling floor laughing ।
2                                      মানুহে যেতিয়া নিজৰ জীৱন টো নিজৰ কাৰণে সমাজৰ কাৰণে দেশৰ কাৰণে পৃথিৱীৰ কাৰণে কিমান মূল্যবান বুজি নাপায় তেতিয়া এনেকুৱা হয়
3                                                                                                                   কতি খন কোন দিনা দেখা পাও তাকে বাত চায় আছো মই
4       কাপোৰ খন চাইকেল অহা মানুহজনৰ মুখত দলিয়াব লাগেনে বেছেৰাটো গুচি গল সেইকাৰণে বেয়া পায়। rolling floor laughing rolling floor laughing rolling floor laughing
                                                                                  ...                                                                            
4031                        

In [ ]:
asa_test['clean_text'] = asa_test['text'].apply(lambda x:emo(x))
asa_test['clean_text'] = asa_test['clean_text'].apply(lambda x:remove_urls(x))
asa_test['clean_text'] = asa_test['clean_text'].apply(lambda x:clean_text(x))

In [ ]:
asa_test['clean_text']

0                                               তথাপিও ভোট বিজেপিকেই দিয়া হব সিধা কথা লাগিলে ভিক্ষাৰি হম তথাপিও ভোট বিজেপিকেই দিম
1                                                                                                মদ খাই নিজৰ কেৰিয়াৰ ধ্বংস কৰিছে।
2                                                            বিপ্লৱৰ কাইঁটীয়া পথ ইমান সহজ নহয় বাপুহঁত।সহজলভ্য জীৱন লাগে তোমালোকক
3                                                                     ক্লা ফকটীয়া ফাল্টু ঘোঁ‌ৰা চহিচ । কাম নাই ষ্টেচনত ঝাড়ু মাৰ গৈ
4       যিয়ে কোচবিহাৰ ৰাজ্যৰ বিৰোধিতা কৰে সি মোগলৰ বংশধৰ কোচবিহাৰ ৰাজ্য হোৱাৰ লগে লগে বস্তুাত ভৰাই ব্ৰহ্মপুত্ৰ নদীত পেলাই দিয়া হব।
                                                                   ...                                                            
1004                                                                                            ভাই বিয়া চিয়া পাতি আমাক পাহৰি গৈছা
1005                                                     সুদুৰ ভাই কোবাট মুৰি আচেলি

Transliteration using libindic

In [ ]:
!git clone https://github.com/libindic/Transliteration.git

fatal: destination path 'Transliteration' already exists and is not an empty directory.


In [ ]:
pip install --upgrade pip

In [ ]:
!python -m pip install --user virtualenv

  Obtaining dependency information for virtualenv from https://files.pythonhosted.org/packages/17/8d/6989e5dcd812520cbf9f31be2b08643ae3a895586601bbab501df8ed6e54/virtualenv-20.24.3-py3-none-any.whl.metadata
  Obtaining dependency information for distlib<1,>=0.3.7 from https://files.pythonhosted.org/packages/43/a0/9ba967fdbd55293bacfc1507f58e316f740a3b231fc00e3d86dc39bc185a/distlib-0.3.7-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 15.9 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
!python -m virtualenv env

created virtual environment CPython3.10.12.final.0-64 in 504ms
  creator CPython3Posix(dest=/content/env, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==23.2.1, setuptools==68.0.0, wheel==0.41.1
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [ ]:
!source env/bin/activate

In [ ]:
pwd

'/content'

In [ ]:
!pip install -r '/content/Transliteration/requirements.txt'

  Cloning https://github.com/libindic/normalizer.git to /tmp/pip-install-dirbql00/libindic-normalizer_a75108a22b744bd29b272c07ab55a916
  Running command git clone --filter=blob:none --quiet https://github.com/libindic/normalizer.git /tmp/pip-install-dirbql00/libindic-normalizer_a75108a22b744bd29b272c07ab55a916
  Resolved https://github.com/libindic/normalizer.git to commit 88447f611e44ddfc9e9c6696650671afc10c25ae
  Preparing metadata (setup.py) ... done
Ignoring repoze.lru: markers 'python_version < "3"' don't match your environment


In [ ]:
!pip install  '/content/Transliteration/.'

Processing ./Transliteration
  Preparing metadata (setup.py) ... done
  Created wheel for libindic.transliteration: filename=libindic.transliteration-0.4.1-py3-none-any.whl size=851872 sha256=881d365b776eec0d3b55688c62758c9dfe75392aef8cbcf582efa0afde27cbb1
  Stored in directory: /tmp/pip-ephem-wheel-cache-qqpja8_s/wheels/94/ea/8b/a1558e8a38a7980bb506204694fd26148d79d71167f025eb79
Successfully built libindic.transliteration
  Attempting uninstall: libindic.transliteration
    Found existing installation: libindic.transliteration 0.4.1
    Uninstalling libindic.transliteration-0.4.1:
      Successfully uninstalled libindic.transliteration-0.4.1


In [ ]:
from libindic.transliteration import getInstance

In [ ]:
ldata=asa_train.clean_text
ldata=ldata.tolist()

In [ ]:
ldata=asa_test.clean_text
ldata=ldata.tolist()

In [ ]:
len(ldata)

1009

In [ ]:
fin=[]
for i in ldata:
  t = getInstance()
  t_text = t.transliterate(str(i), "en_IN")
  fin.append(t_text)
len(fin)

1009

In [ ]:
df=pd.DataFrame(fin)
df=df.rename(columns={0:'texttranslit'})
df.head(20)

,texttranslit
0,thathaapio bhoot bijepikei diy഼aa hab sidhaa kathaa laagile bhikshaa൰i ham thathaapio bhoot bijepikei dim
1,mad khaai nij൰ ke൰iy഼aa൰ dhbams k൰ichhe೤
2,bipl൱൰ kaaiഁateey഼aa path imaan sahaj nahay഼ baapuhഁath೤asahajalabhy jee൱an laage thoomaalookak
3,klaa phakateeൟaa phaaltu ghooഁ‌൰aa chahich ।kaam naai shtechanath jhhaa൜u maa൰ gai
4,yiൟe koochabihaa൰ ൰aajy൰ bi൰oodhithaa k൰e si moogal൰ bamsadh൰ koochabihaa൰ ൰aajy hoo൱aa൰ lage lage basthuaath bh൰aai b൰ahmaputh൰ nadeeth pelaai diൟaa hab೤
5,sachaakai bhag൱aan൰ aan etaa ൰oop೤ chikiൎasak sakal naathaakile kimaan maanuh൰ mrithyu hal heഁathen bhaa൱ilei bhay഼ laage೤
6,ethiൟaa keitaa maan ൰aajanaithik dale bhoot൰ sbaarthath hindu dharmaa൱alee look sakalak santhasth k൰ib൰ kaa൰ane aas൰aa aa൰ambh k൰aa yen laagichhe kaam nahab bahuth palam gal ৷
7,dh൰m bhagabaan൰ mandi൰ath kih൰ tiketh laage dh൰masthaan ne by൱asaai ।
8,maanuhe ke൱al ke൱al maath൰ etaa l൰aa baa ejanee chhoo൱aalee൰ p൰em൰ kaa൰ane janm g൰ahan k൰e neki
9,baakee chab yahannaame yaa


In [ ]:
X_train=asa_train['clean_text']
X_test=asa_test['clean_text']
y_train=asa_train['task_1']

In [ ]:
tfidf_vect = TfidfVectorizer(analyzer='char', ngram_range=(1,3))
tfidf_vect.fit(X_train)
X_train = tfidf_vect.transform(X_train)
X_test = tfidf_vect.transform(X_test)

In [ ]:
from sklearn.svm import SVC
svm = SVC()

svm .fit(X_train, y_train)

y_pred = svm.predict(X_test)


In [ ]:
#Submission
# to create .csv file consisting of tweet ids and predicted labels
y_pred_df= pd.DataFrame(data=y_pred, columns=['y_pred'])
Submisssion_assam = pd.DataFrame()
Submisssion_assam['S. No.'] = asa_test['S. No.']
Submisssion_assam['task_1'] = y_pred_df
Submisssion_assam.to_csv('/content/drive/MyDrive/MUCS_test_assam__run14.csv',index = None)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import VotingClassifier
from sklearn.svm import LinearSVC

clf1 = LogisticRegression(random_state=1)
clf2 = BernoulliNB()
clf3 = LinearSVC(penalty='l2',C=1.0)

eclf = VotingClassifier(estimators=[('LR', clf1), ('BNB', clf2),
('SVC', clf3)], voting='hard')
eclf = eclf.fit(X_train, y_train)
y_pred1= eclf.predict(X_test)
print(y_pred1)

['HOF' 'HOF' 'NOT' ... 'HOF' 'HOF' 'HOF']


In [ ]:
#Submission
# to create .csv file consisting of tweet ids and predicted labels
y_pred_df= pd.DataFrame(data=y_pred1, columns=['y_pred1'])
Submisssion_assam = pd.DataFrame()
Submisssion_assam['S. No.'] = asa_test['S. No.']
Submisssion_assam['task_1'] = y_pred_df
Submisssion_assam.to_csv('/content/drive/MyDrive/MUCS_test_asam_run15.csv',index = None)